In [3]:
# Konfigurasi Spark
import os
import sys

# 1. Lokasi di mana Spark diinstal
spark_path = "E:\smt8\spark"

# 2. Menentukan environment variable
os.environ['SPARK_HOME'] = spark_path


# 3. Download winutils dari https://github.com/steveloughran/winutils/blob/master/hadoop-2.6.0/bin/winutils.exe?raw=true
#   dan letakkan di dalam folder D:\spark\bin\
#   Lokasi winutils.exe
os.environ['HADOOP_HOME'] = spark_path

# 4. Lokasi Python yang dijalankan --> punya Anaconda
#    Apabila Python yang diinstall hanya Anaconda, maka tidak perlu menjalankan baris ini.
os.environ['PYSPARK_PYTHON'] = sys.executable

# 5. Konfigurasi path library PySpark
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "test")

print sc

In [4]:
from numpy import array
from math import sqrt

In [5]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [6]:
# Load and parse the data
data = sc.textFile("E:\smt8\spark\data\mllib\scrubbed.csv")
print data

E:\smt8\spark\data\mllib\scrubbed.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [12]:
datanoheader = data.filter(lambda x:"datetime,city" not in x)
datanoheader.collect()

[u'10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit,4/27/2004,29.8830556,-97.9411111',
 u'10/10/1949 21:00,lackland afb,tx,,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX.  Lights racing across the sky &amp; making 90 degree turns on a dime.,12/16/2005,29.38421,-98.581082',
 u'10/10/1955 17:00,chester (uk/england),,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 England,1/21/2008,53.2,-2.916667',
 u'10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving the only Edna theater at about 9 PM&#44...we had our bikes and I took a different route home,1/17/2004,28.9783333,-96.6458333',
 u'10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/attack aircraft on a solo night exercise&#44 I was at 50&#44000&#39 in a &quot;clean&quot; aircraft (no ordinan,1/2

In [13]:
datasplitted = datanoheader.map(lambda x:x.split(','))
datasplitted.collect()

[[u'10/10/1949 20:30',
  u'san marcos',
  u'tx',
  u'us',
  u'cylinder',
  u'2700',
  u'45 minutes',
  u'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit',
  u'4/27/2004',
  u'29.8830556',
  u'-97.9411111'],
 [u'10/10/1949 21:00',
  u'lackland afb',
  u'tx',
  u'',
  u'light',
  u'7200',
  u'1-2 hrs',
  u'1949 Lackland AFB&#44 TX.  Lights racing across the sky &amp; making 90 degree turns on a dime.',
  u'12/16/2005',
  u'29.38421',
  u'-98.581082'],
 [u'10/10/1955 17:00',
  u'chester (uk/england)',
  u'',
  u'gb',
  u'circle',
  u'20',
  u'20 seconds',
  u'Green/Orange circular disc over Chester&#44 England',
  u'1/21/2008',
  u'53.2',
  u'-2.916667'],
 [u'10/10/1956 21:00',
  u'edna',
  u'tx',
  u'us',
  u'circle',
  u'20',
  u'1/2 hour',
  u'My older brother and twin sister were leaving the only Edna theater at about 9 PM&#44...we had our bikes and I took a different route home',
  u'1/17/2004',
 

In [14]:
datalatlng = datasplitted.map(lambda x:x[-2:])
datalatlng.collect()

[[u'29.8830556', u'-97.9411111'],
 [u'29.38421', u'-98.581082'],
 [u'53.2', u'-2.916667'],
 [u'28.9783333', u'-96.6458333'],
 [u'21.4180556', u'-157.8036111'],
 [u'36.595', u'-82.1888889'],
 [u'51.434722', u'-3.18'],
 [u'41.1175', u'-73.4083333'],
 [u'33.5861111', u'-86.2861111'],
 [u'30.2947222', u'-82.9841667'],
 [u'33.9163889', u'-118.3516667'],
 [u'35.2333333', u'-82.7344444'],
 [u'40.6686111', u'-73.5275'],
 [u'37.1536111', u'-83.7619444'],
 [u'35.8238889', u'-80.2536111'],
 [u'36.8430556', u'-83.3219444'],
 [u'42.5377778', u'-83.2330556'],
 [u'41.3252778', u'-72.1936111'],
 [u'32.364167', u'-64.678611'],
 [u'42.3916667', u'-71.5666667'],
 [u'51.5', u'-3.2'],
 [u'38.1055556', u'-98.6597222'],
 [u'32.8544444', u'-79.975'],
 [u'45.5827778', u'-122.3522222'],
 [u'51.783333', u'-0.783333'],
 [u'29.4238889', u'-98.4933333'],
 [u'38.2541667', u'-85.7594444'],
 [u'40.7008333', u'-73.7133333'],
 [u'41.7002778', u'-73.9213889'],
 [u'53.970571', u'-111.689885'],
 [u'32.7152778', u'-117.1563

In [16]:
datasaved = datalatlng.map(lambda line: array([x for x in line]))
datafix = datasaved.map(lambda line: ' '.join(line[0:2]))
datafix.collect()

[u'29.8830556 -97.9411111',
 u'29.38421 -98.581082',
 u'53.2 -2.916667',
 u'28.9783333 -96.6458333',
 u'21.4180556 -157.8036111',
 u'36.595 -82.1888889',
 u'51.434722 -3.18',
 u'41.1175 -73.4083333',
 u'33.5861111 -86.2861111',
 u'30.2947222 -82.9841667',
 u'33.9163889 -118.3516667',
 u'35.2333333 -82.7344444',
 u'40.6686111 -73.5275',
 u'37.1536111 -83.7619444',
 u'35.8238889 -80.2536111',
 u'36.8430556 -83.3219444',
 u'42.5377778 -83.2330556',
 u'41.3252778 -72.1936111',
 u'32.364167 -64.678611',
 u'42.3916667 -71.5666667',
 u'51.5 -3.2',
 u'38.1055556 -98.6597222',
 u'32.8544444 -79.975',
 u'45.5827778 -122.3522222',
 u'51.783333 -0.783333',
 u'29.4238889 -98.4933333',
 u'38.2541667 -85.7594444',
 u'40.7008333 -73.7133333',
 u'41.7002778 -73.9213889',
 u'53.970571 -111.689885',
 u'32.7152778 -117.1563889',
 u'38.7583333 -104.7425',
 u'29.7630556 -95.3630556',
 u'32.7833333 -96.8',
 u'42.9955556 -71.4552778',
 u'-38.662334 178.017649',
 u'29.7630556 -95.3630556',
 u'44.7630556 -85.62

In [19]:
parsedData = datafix.map(lambda line: array([float(x.strip()) for x in line.split(' ')]))
parsedData.collect()

[array([ 29.8830556, -97.9411111]),
 array([ 29.38421 , -98.581082]),
 array([ 53.2     ,  -2.916667]),
 array([ 28.9783333, -96.6458333]),
 array([  21.4180556, -157.8036111]),
 array([ 36.595    , -82.1888889]),
 array([ 51.434722,  -3.18    ]),
 array([ 41.1175   , -73.4083333]),
 array([ 33.5861111, -86.2861111]),
 array([ 30.2947222, -82.9841667]),
 array([  33.9163889, -118.3516667]),
 array([ 35.2333333, -82.7344444]),
 array([ 40.6686111, -73.5275   ]),
 array([ 37.1536111, -83.7619444]),
 array([ 35.8238889, -80.2536111]),
 array([ 36.8430556, -83.3219444]),
 array([ 42.5377778, -83.2330556]),
 array([ 41.3252778, -72.1936111]),
 array([ 32.364167, -64.678611]),
 array([ 42.3916667, -71.5666667]),
 array([ 51.5,  -3.2]),
 array([ 38.1055556, -98.6597222]),
 array([ 32.8544444, -79.975    ]),
 array([  45.5827778, -122.3522222]),
 array([ 51.783333,  -0.783333]),
 array([ 29.4238889, -98.4933333]),
 array([ 38.2541667, -85.7594444]),
 array([ 40.7008333, -73.7133333]),
 array([

In [20]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel

In [21]:
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

In [22]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [23]:
WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 1786992.38795


In [24]:
clusterData = parsedData.map(clusters.predict)

In [25]:
X = parsedData.collect()

In [26]:
label = clusterData.collect()

In [27]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for point in range(len(label)):
    if (label[point]==1):
        color = "r";
    else:
        color = "g";
    lines = plt.plot(X[point][0], X[point][1], 'ro')
    plt.setp(lines, color=color, linewidth=2.0)

In [ ]:
center = clusters.clusterCenters
print center

In [ ]:
for centroid in range(len(center)):
    lines = plt.plot(center[centroid][0],center[centroid][1],'bx')
    plt.setp(lines, color='b', markersize=20)

In [ ]:
plt.show()